<a href="https://colab.research.google.com/github/EdoardoMorucci/Plant-Leaves-Search-Engine---MIRCV/blob/main/model_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
This notebook describes the fine-tuning process of Convolutional Neural Network using as Base Network DenseNet

# Local download of the dataset

In [7]:
! pip install -q kaggle

from google.colab import files
_ = files.upload()

! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [8]:
! kaggle datasets download -d davidedemarco/healthy-unhealthy-plants-dataset-segmented --unzip

100% 641M/642M [00:11<00:00, 49.4MB/s]
100% 642M/642M [00:11<00:00, 58.3MB/s]


# Per i Raga
Raga miraccomando rinominate la directory con il dataset così "Healthy-and-Unhealthy-Plants-Dataset-Segmented" per evitare esplosioni

# Connection to Google Drive

In [10]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


# Import

In [2]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Data Preparation


The dataset is on Google Drive and the dataset directory has the structure:

```
dataset/
  class_1/
    image_1.jpg
    image_2.jpg
    ...
  class_2/
    image_3.jpg
    image_4.jpg
    ...
  ...
  ...
  class_n/
    ...
```

To train and test the model, we need three subsets: train, test and validation. To split the dataset, we use the [split-folder](https://pypi.org/project/split-folders/) package.

In [5]:
!pip install split-folders tqdm

We need to check if the hardware accelaration is enabled, since training a CNN on a CPU could be infeasible.

In [4]:
#check hardware acceleration
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


We define the costants with the directory of the dataset and the directory where the datasplits are created. In addition we define the image size and the batch size.

In [3]:
BASE_DIR = "gdrive/Shareddrives/MIRCV-PlantLeavesSearchEngine/"
DATA_DIR = '/content/Healthy-and-Unhealthy-Plants-Dataset-Segmented'
SETS_DIR = '/content/healthy-unhealthy-plants-sets'
MODEL_DIR = '/content/model'

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 64
N_CLASSES = 14

We need to create data splits. The dataset will be divided 80% in training set, 10% in validation set and 10% in test set.

In [11]:
import splitfolders
# split data
splitfolders.ratio(DATA_DIR, output=SETS_DIR, seed=123, ratio=(0.8, 0.1, 0.1), group_prefix=None)



Copying files: 0 files [02:36, ? files/s]


Copying files: 636 files [00:00, 4110.35 files/s]

Copying files: 1337 files [00:00, 5501.66 files/s]

Copying files: 2134 files [00:00, 6503.42 files/s]

Copying files: 2930 files [00:00, 7037.75 files/s]

Copying files: 3656 files [00:00, 6994.01 files/s]

Copying files: 4370 files [00:00, 6311.64 files/s]

Copying files: 5070 files [00:00, 6513.08 files/s]

Copying files: 5815 files [00:00, 6789.35 files/s]

Copying files: 6549 files [00:00, 6952.24 files/s]

Copying files: 7254 files [00:01, 5385.17 files/s]

Copying files: 8009 files [00:01, 5919.07 files/s]

Copying files: 8791 files [00:01, 6415.04 files/s]

Copying files: 9570 files [00:01, 6786.86 files/s]

Copying files: 10285 files [00:01, 6039.98 files/s]

Copying files: 10928 files [00:01, 5529.18 files/s]

Copying files: 11513 files [00:01, 5176.80 files/s]

Copying files: 12054 files [00:02, 4930.84 files/s]

Copying files: 12563 files [00:02, 3781.27 files/s]

Copying files:

Now we need to create the Dataset objects from the sets directory. We use the [image_dataset_from_directory](https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory) function provided by Keras. An example of use of this library can be found on the official documentation provided by Keras ([here](https://keras.io/examples/vision/image_classification_from_scratch/)).

In [12]:

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    SETS_DIR + '/train',
    labels='inferred', #the label of the dataset is obtained by the name of the directory
    seed=123,
    shuffle=True,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    SETS_DIR + '/val',
    labels='inferred', #the label of the dataset is obtained by the name of the directory
    seed=123,
    shuffle=True,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
)
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    SETS_DIR + '/test',
    labels='inferred', #the label of the dataset is obtained by the name of the directory
    seed=123,
    shuffle=True,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
)


Found 57623 files belonging to 14 classes.
Found 7195 files belonging to 14 classes.
Found 7216 files belonging to 14 classes.


The images needs to be preprocessed before going in input to the CNN DenseNet. We use the function [tf.keras.applications.densenet.preprocess_input](https://www.tensorflow.org/api_docs/python/tf/keras/applications/densenet/preprocess_input) to preprocess the image. In addition we add the batch dimension.

In [13]:
def preprocess(images, labels):
  images = tf.keras.applications.densenet.preprocess_input(images)
  return images, labels
  
#preprocessing of the images in all the set
train_ds = train_ds.map(preprocess)
val_ds = val_ds.map(preprocess)
test_ds = test_ds.map(preprocess)

# Training

The CNN used has base network is DenseNet. Since we want to fine-tune the network. We remove the fully-connected layer on top and later we will add an output layer with 14 neurons (1 for each class we want to predict).

In [14]:
pretrained_model = tf.keras.applications.DenseNet121(
    input_shape = (224, 224, 3),
    weights="imagenet",
    include_top=False,  # do not include the pretrained layers implementing the imagenet classifier
)

# freezes weights of all levels of the pre-trained network
pretrained_model.trainable = False 

#pretrained_model.summary()

29097984/29084464 [==============================] - 0s 0us/step


On top of the base network we apply global average pooling and we add an hidden classifier with 256 neurons. The last layer of the network is the output classification layer, with 1 neuron for each class and with softmax as activation function.

In [15]:
from tensorflow.keras import layers as L

x = pretrained_model.output

# add a global average pooling
x = L.GlobalAveragePooling2D(name='gap')(x)
x = L.Flatten(name='flatten')(x)

# add a fully-connected layer (Dense) of 256 neurons with name='classifier_hidden'
x = L.Dense(256,activation='relu', name='classifier_hidden')(x)

# add output classification layer with n_classes outputs and softmax activation
x = L.Dense(N_CLASSES, activation='softmax')(x)
new_output = x

model = tf.keras.models.Model(inputs=pretrained_model.input, outputs=new_output, name='healthy_and_unhealty_plants_classifier')

#model.summary()

To prevent huge gradients coming from the newly initialized layers from destroying the weights in the pretrained layers  we will initially freeze the layers of the base network and train only new layers. As optimizers we use Adam.

In [16]:
learning_rate=0.005
epochs=20

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

model.compile(optimizer,
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])
callbacks = [
  # early stopping
      tf.keras.callbacks.EarlyStopping(
          monitor='val_loss', 
          patience=2,
          restore_best_weights=True),

  # checkpoint best model 
  tf.keras.callbacks.ModelCheckpoint(
    filepath=MODEL_DIR + "/healthy_and_unhealty_plants_classifier",
    save_weights_only=True,
    monitor='accuracy',
    mode='max',
    save_best_only=True
  ),
]

train_ds_shuffle = train_ds.shuffle(123)  # shuffles data each epoch

# train the model
history = model.fit(
  train_ds_shuffle,
  validation_data=val_ds,
  epochs = epochs,  
  callbacks=callbacks,
  batch_size=BATCH_SIZE,
  verbose=1
)


Epoch 1/20
  4/901 [..............................] - ETA: 2:11:59 - loss: 3.5564 - accuracy: 0.3047

KeyboardInterrupt: ignored

In [79]:
model.save('/content/model')

INFO:tensorflow:Assets written to: /content/model/assets


In [82]:
model = keras.models.load_model('/content/model')

In [83]:
model.summary()

Model: "healthy_and_unhealty_plants_classifier"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_12 (ZeroPadding  (None, 230, 230, 3)  0          ['input_8[0][0]']                
 2D)                                                                                              
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d_12[0][0]']      
                                )                            

Now we need to unfreeze layers of the base network and reperforms training in order to fine-tune the network. Since this operation is very expensive in terms of resources, we decided to unfreeze only the last two blocks of the network. As we cans see from the output of the next cell, the number of traainable parameters of the network increased from 265,998 to 589,582 

In [84]:
trainable = False
for layer in model.layers:
    if layer.name == 'conv5_block15_0_bn':
        trainable = True
    is_bn = layer.name.endswith('bn')  # keep BatchNorm freezed, good practice for finetuned models
    if not is_bn:
      layer.trainable = trainable

model.summary()

Model: "healthy_and_unhealty_plants_classifier"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_12 (ZeroPadding  (None, 230, 230, 3)  0          ['input_8[0][0]']                
 2D)                                                                                              
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d_12[0][0]']      
                                )                            

Now we perform fine-tuning.

In [85]:
learning_rate = 0.01
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

model.compile(optimizer,
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])

# train the model
history = model.fit(
  train_ds_shuffle,
  validation_data=val_ds,
  epochs = epochs,  
  callbacks=callbacks,
  batch_size=BATCH_SIZE,
  verbose=1
)

Epoch 1/2
28812/28812 [==============================] - 1636s 56ms/step - loss: 0.5314 - accuracy: 0.8701 - val_loss: 133.3201 - val_accuracy: 0.1233
Epoch 2/2
28812/28812 [==============================] - 1614s 56ms/step - loss: 0.4518 - accuracy: 0.8810 - val_loss: 115.5528 - val_accuracy: 0.1819


# Dubbi da discutere insieme appassionatamente


*   Quali sono i layer da unfreezare durante il fine tuning?
* I layer copiati dal notebook perchè esistono?
*   Learning rate ed epochs tenendo conto del tempo di training
* Data augmentation (o anche dropout magari)
* Early stopping e momentum
* Quale ottimizzatore utilizzare

